In [1]:
import os
import pandas as pd
from datetime import datetime as dt

import altair as alt
import numpy as np
import matplotlib.pyplot as plt

from pprint import pprint
import datetime

In [ ]:
file='./data/StreamingHistory.json'

#### Grabbing streaming data starting in 2020

In [ ]:
stream=pd.read_json(file)

#'endTime' is timestamp that song ended - taking this and 'msPlayed' (miliseconds played) to calculate the timestamp that song started.
#this will handle any songs that started before midnight
stream['endTime']=pd.to_datetime(stream['endTime'])
time_delta_0=np.array([pd.Timedelta(i) for i in stream['msPlayed']])
start_0=[]
for time_i_0, delta_i_0 in zip(stream['endTime'],time_delta_0):
        start_0.append(time_i_0-delta_i_0)
stream['startTime']=start_0

#fitler out all data that was not in 2020
start_2020=dt.strptime('2020-01-01 00:01', '%Y-%m-%d %H:%M')
start_2021=dt.strptime('2021-01-01 00:01', '%Y-%m-%d %H:%M')

stream_2020=stream[(stream['startTime']>=start_2020)&(stream['startTime']<start_2021)]

---
### CREATING ARTIST METADATA DATAFRAME/GETTING GENRES
#### See Spotify API notebook Metadata df build

In [ ]:
ARTISTS_METADATA=pd.read_json('./data/artists_metadata.json')

---
##### Report the number of unique genres and the number of times they appear

In [ ]:
UNIQUE_GENRES=[genre_i for genre_list in ARTISTS_METADATA['GENRES'] for genre_i in genre_list]
UNIQUE_GENRES=np.unique(UNIQUE_GENRES, return_counts=True)

GENRE_COUNTS=dict(zip(UNIQUE_GENRES[0], UNIQUE_GENRES[1]))
GENRE_COUNTS=sorted(GENRE_COUNTS.items(), key= lambda x:x[1], reverse=True)


---

### CREATE DF FOR GENRE COUNTS TO ADD TIME TO

In [ ]:
## empty df 
SEC_GENRE_COUNTS=pd.DataFrame(index=UNIQUE_GENRES[0],columns=(stream_2020['endTime'].unique()))
SEC_GENRE_COUNTS=SEC_GENRE_COUNTS.fillna(0.0)

---

### COUNTING STREAMS

In [ ]:
#loop through each song, take the artist and time played, then add time to the associated genres
for sec_i in (stream_2020.index.unique()):
   # print(sec_i)
    SEC_DF=stream_2020[stream_2020.index==sec_i]

    for art_i, stream_time_i in zip(SEC_DF['artistName'],SEC_DF['msPlayed']):

        
        # Connect to artist metadata to grab the genres associated with them
        art_i_genres=SEAN_ARTISTS_METADATA.loc[art_i]['GENRES']
        for genre_i in art_i_genres:
            #add the time played

            SEC_GENRE_COUNTS.loc[genre_i, sec_i]+=stream_time_i


In [ ]:
# Correct case of genre titles
SEC_GENRE_COUNTS.index=[i.title() for i in SEC_GENRE_COUNTS.index]

In [ ]:
#SAVE
SEC_GENRE_COUNTS.to_csv('./genre_listen_counts.csv')
